# Análisis del Riesgo de Obesidad y problemas CardioVasculares


# Carga de librerías y configuraciones

In [ ]:
# warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Carga funciones propias reutilizables  de librerías
import os as so
import sys
utils_path = so.path.join(so.getcwd(), '..', 'utils')
sys.path.append(utils_path)

In [ ]:
class PATH():
    RAW = '../data/raw//'
    PROCESSED= '../data/processed//'
    MODELS = '../models//'
    REPORTS = '../reports//'


In [ ]:
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
# pandas config
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# seaborn config
colors_palette=sns.color_palette('colorblind')

In [ ]:

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [ ]:
from scipy.stats import skew, kurtosis
from scipy.stats import shapiro

In [ ]:
import python_clustering

In [ ]:
from BorutaShap import BorutaShap

In [ ]:
from utils import *

# EDA

## Carga datos


In [ ]:
df_train = pd.read_csv(PATH.RAW + 'train.csv')
df_test = pd.read_csv(PATH.RAW +'test.csv')

## Significado de las columnas
Los datos consisten en la estimación de los niveles de obesidad en personas de los países de México, Perú y Colombia, con edades entre 14 y 61 años y diversos hábitos alimenticios y condiciones físicas. Los datos se recopilaron utilizando una plataforma web con una encuesta donde usuarios anónimos respondieron cada pregunta, luego la información fue procesada obteniendo 17 atributos y 2111 registros.

Los atributos relacionados con los hábitos alimenticios son: 
- Consumo frecuente de alimentos altos en calorías (FAVC)
- Frecuencia de consumo de vegetales (FCVC)
- Número de comidas principales (NCP)
- Consumo de alimentos entre comidas (CAEC)
- Consumo de agua diario (CH20)
- Consumo de alcohol (CALC)

Los atributos relacionados con la condición física son: 
- Monitoreo del consumo de calorías (SCC)
- Frecuencia de actividad física (FAF)
- Tiempo utilizando dispositivos tecnológicos (TUE)
- Transporte utilizado (MTRANS)

Variables obtenidas: Género, Edad, Altura y Peso.

Los valores de NObesidad son:

- Bajo peso: Menos de 18.5
- Normal: 18.5 a 24.9
- Sobrepeso: 25.0 a 29.9
- Obesidad I: 30.0 a 34.9
- Obesidad II: 35.0 a 39.9
- Obesidad III: Más de 40

Los datos contienen datos numéricos y datos continuos, por lo que pueden ser utilizados para análisis basados en algoritmos de clasificación, predicción, segmentación y asociación.

## Diccionarios

In [ ]:
numericas

In [ ]:
diccionario_columnas = {
    'FAVC': 'Consumo frecuente de alimentos altos en calorías',
    'FCVC': 'Frecuencia de consumo de vegetales',
    'NCP': 'Número de comidas principales',
    'CAEC': 'Consumo de alimentos entre comidas',
    'CH20': 'Consumo de agua diario',
    'CALC': 'Consumo de alcohol',
    'SCC': 'Monitoreo del consumo de calorías',
    'FAF': 'Frecuencia de actividad física',
    'TUE': 'Tiempo utilizando dispositivos tecnológicos',
    'MTRANS': 'Transporte utilizado',
    'Age':'Edad',
    'Height': 'Altura',
    'Weight':'Peso',
    'SMOKE':'Fumador',
    'Gender':'Género',
    'IMC':'Indice de Masa Corporal',
    'HA': 'Hábitos alimienticios',
    'UT': 'Uso de Tecnología',
    'family_history_with_overweight':'Historial familiar con sobrepeso'
}


## Target
   

In [ ]:
target = "NObeyesdad"

In [ ]:
dict_target={'Insufficient_Weight': 0,
 'Normal_Weight': 1,
 'Overweight_Level_I': 2,
 'Overweight_Level_II': 3,
  'Obesity_Type_I': 4,
 'Obesity_Type_II': 5,
 'Obesity_Type_III': 6,}


In [ ]:
dictrev_target={v:k for k,v in  dict_target.items()}


1.3 Vistazo rápido

In [ ]:
df=df_train.copy()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe(include='all').T

In [ ]:
data_report(df)

Highlights: Sin missings, no tiene mucha cardinalidad.

In [ ]:
contains_inf = df.isin([np.inf, -np.inf]).any().any()
contains_inf

No hay infinitos

In [ ]:
duplicados = df.duplicated().any()
duplicados

Sin duplicados

In [ ]:
categoricas = df.columns[df.dtypes=="object"].tolist()
numericas = df.columns[df.dtypes!="object"].tolist()

In [ ]:
plot_tidy_categorical(df, categoricas, target)

In [ ]:
plot_tidy(df, numericas, target)

# Problema Machine Learning
   

In [ ]:
df[target].value_counts() / len(df[target])

Al examinar la distribución del objetivo, podemos deducir lo siguiente:

- Se trata de un problema de clasificación múltiple, con 7 clases.
- Las clases están distribuidas de manera diferente, pero no hay diferencias extremas en sus proporciones (probabilidad media).
- Sin embargo, la clase más frecuente (Obesidad_Tipo_III) tiene casi el doble de probabilidad que la menos frecuente (Sobrepeso_Nivel_I). Por lo tanto, al definir la estrategia de validación cruzada, será estratificada  para que las diferentes probabilidades previas se reflejen lo más exactamente posible también en el conjunto de prueba.

# Feature Engineering


![](https://www.stylecraze.com/wp-content/themes/buddyboss-child/images/man-body-mass-index-vector.jpg)

Índice de Masa Corporal (IMC):
Utilizando las características de 'Altura' y 'Peso'. El IMC, una métrica ampliamente reconocida, indica la obesidad al proporcionar una representación más precisa de la relación entre el peso y la altura de un individuo.

Hábitos Alimenticios (HA):
La combinación de 'FCVC' (Frecuencia de consumo de vegetales) y 'NCP' (Número de comidas principales) creó la característica 'Hábitos_Alimenticios'. Esta característica busca encapsular los patrones dietéticos generales, considerando tanto la frecuencia de consumo de vegetales como el número de comidas principales.

Puntuación de Uso de Tecnología (UT):
Se creó una puntuación integral ponderando la frecuencia de uso de la tecnología ('UT') por la edad del individuo. La puntuación resultante de 'Uso_de_Tecnología' tiene como objetivo cuantificar el tiempo promedio que se pasa utilizando la tecnología en relación con la edad de la persona, proporcionando una perspectiva matizada sobre los hábitos tecnológicos.

Hay columnas que tienen orden y que deben convertirse a variables discretas. Las mapeo también.

In [ ]:
dict_CAEC={
    'no': 0,
    'Sometimes': 1,
    'Frequently': 2,
    'Always': 3}
dict_CALC={
    'no': 0,
    'Sometimes': 1,
    'Frequently': 2,
    'Always': 3}

In [ ]:
def Feature_Engineering(df):
    df.set_index('id', inplace=True)
    df['IMC'] = df['Weight'] / (df['Height'] ** 2)
    df['HA'] = df['FCVC'] * df['NCP']
    df['UT'] = df['TUE'] / df['Age']
    df['CALC']=df['CALC'].map(dict_CALC)
    df['CAEC']=df['CAEC'].map(dict_CAEC)
    return df

In [ ]:
df_train = Feature_Engineering(df_train)
df_test = Feature_Engineering(df_test)
df = Feature_Engineering(df)
numericas = df.columns[df.dtypes!="object"].tolist()

In [ ]:
diccionario_columnas = {
    'FAVC': 'Consumo frecuente de alimentos altos en calorías',
    'FCVC': 'Frecuencia de consumo de vegetales',
    'NCP': 'Número de comidas principales',
    'CAEC': 'Consumo de alimentos entre comidas',
    'CH20': 'Consumo de agua diario',
    'CALC': 'Consumo de alcohol',
    'SCC': 'Monitoreo del consumo de calorías',
    'FAF': 'Frecuencia de actividad física',
    'TUE': 'Tiempo utilizando dispositivos tecnológicos',
    'MTRANS': 'Transporte utilizado',
    'Age':'Edad',
    'Height': 'Altura',
    'Weight':'Peso',
    'SMOKE':'Fumador',
    'Gender':'Género',
    'IMC':'Indice de Masa Corporal',
    'HA': 'Hábitos alimienticios',
    'UT': 'Uso de Tecnología',
    'family_history_with_overweight':'Historial familiar con sobrepeso'
}


In [ ]:
df.head()

# Análisis univariante


In [ ]:
categoricas = df.columns[df.dtypes=="object"].tolist()
numericas = df.columns[df.dtypes!="object"].tolist()

In [ ]:
plot_horizontal_catplot(df, categoricas, diccionario_columnas)

In [ ]:
df[target]=df[target].map(dict_target)

In [ ]:
for col in numericas:
    plot_distribucion(df,col, title=diccionario_columnas.get(col, col))

# Análisis bivariante


In [ ]:
sns.set_theme(style='white')
sns.pairplot(df,kind="reg",diag_kind='kde',plot_kws={'line_kws':{'color':'red'}},corner=True,hue=target)
plt.tight_layout()
plt.show()


In [ ]:
for col in numericas:
    plot_analysis(df, target, col)


# Eliminación de features


In [ ]:
df = drop_cols(df, max_cardi=20, max_miss=30)

# Correlación

In [ ]:
import phik
phik_matrix = df.phik_matrix()

plt.figure(figsize=(10,10))
sns.heatmap(phik_matrix,
            vmin=0,
            vmax=1,
            center=0,
            cmap=sns.diverging_palette(145, 280, s=85, l=25, n=10),
            square=True,
            annot=True,
            linewidths=.5);

# Anomalías y errores


No se han encontrado


# Transformaciones (Encodeing)
    

In [ ]:
def sqrt_transform(X):
    return np.sqrt(X)

def log_transform(X):
    return np.log1p(X)

In [ ]:
classify_distributions(df, threshold=0.05)

In [ ]:
classify_distributions(df.drop(columns=[target]), threshold=0.05)

In [ ]:
import pandas as pd
def Encoder(df, target_col, pca_n_components=8, threshold=0.05):
    dist_class = classify_distributions(df.drop(columns=[target_col]), threshold)
    categorical_columns = list(df.select_dtypes(include=['object']).columns)

    if categorical_columns:
        categorical_pipeline = Pipeline([
            ('onehot', OneHotEncoder(drop='first'))
        ])
    else:
        categorical_pipeline = None

    numeric_pipeline = Pipeline([
        ('transformation', ColumnTransformer([
            ('sqrt_transform', FunctionTransformer(sqrt_transform), [col for col, (dist, _) in dist_class.items() if dist == 'positive_increasing']),
            ('log_transform', FunctionTransformer(log_transform), [col for col, (dist, _) in dist_class.items() if dist == 'positive_decreasing']),
            ('yeojohnson_transform', PowerTransformer(method='yeo-johnson'), [col for col, (dist, _) in dist_class.items() if dist not in ['positive_increasing', 'positive_decreasing']])
        ], remainder='passthrough')),
        ('scaler', StandardScaler())
    ])

    if categorical_pipeline:
        preprocessor = ColumnTransformer([
            ('categorical', categorical_pipeline, categorical_columns),
            ('numeric', numeric_pipeline, [col for col, _ in dist_class.items()])
        ])
    else:
        preprocessor = numeric_pipeline

    final_pipeline = Pipeline([
        ('preprocessor', preprocessor)
    ])

    X = df.drop(columns=[target_col])


    required_columns = set([col for col, _ in dist_class.items()])
    if not required_columns.issubset(X.columns):
        missing_columns = required_columns - set(X.columns)
        raise ValueError(f"Missing columns: {missing_columns}")
  
    return final_pipeline.fit_transform(X)
    


In [ ]:
X_trans=Encoder(df, target_col, pca_n_components=8, threshold=0.05)
X_test=Encoder(df_test, target_col, pca_n_components=8, threshold=0.05)

# 3. División train y test
   

In [ ]:
    # Outliners
def outlier_split(X_trans,y, test_size=0.2):
    
    Xout=X_trans.values
    tasks = python_clustering.Tasks()
    mode = "overall"
    result, detected_outliers, classifiers = tasks.detect_anomalies(
        Xout, methods="all_besides_nn", outliers_fraction=0.05, mode=mode
    )
    tasks.plot_overall_anomaly_classifiers(result, classifiers, show_heatmap=False)
    arr=result[:,-1]
    # Calculate the 95th percentile
    percentile = np.percentile(arr, 95)

    # Create boolean array where values are in the 95th percentile
    boolean_array = arr <= percentile
    X_trans_out=X_trans[boolean_array]
    X_trans_out=pd.DataFrame(data=X_trans_out, columns=X_trans.columns)
    y_out=[y for  i,y in enumerate(y) if boolean_array[i]]
    
    X_train, X_val, y_train, y_val = train_test_split(X_trans_out, y_out, test_size=test_size, random_state=123)

    return X_train, X_val, y_train, y_val

In [ ]:
y = df[target_col]

In [ ]:
X_train, X_val, y_train, y_val=outlier_split(X_trans,y, test_size=0.2)

# 15. Feature Reduction


In [ ]:
selvars=selvars_boruta(df,target)

# 16. Escoger métrica del modelo


## 16.1 Métricas de clasificación

# Baselines


In [ ]:
# Modelos de Clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [ ]:

# Definir los modelos a comparar
modelos = {
    'Logistic Regression': LogisticRegression(solver='liblinear'),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'Support Vector Machine': SVC(gamma='scale', probability=True),
    'Naive Bayes': GaussianNB(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(n_estimators=10),
    'Extra Trees': ExtraTreesClassifier(n_estimators=10),
    #'XGBoost': XGBClassifier(),
    'Random Forest':RandomForestClassifier(n_estimators=10),
    'AdaBoost': AdaBoostClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'LightGBM':LGBMClassifier(),
}


In [ ]:
splits_df = [
    "X_train",
    "X_val",
    "Xtest_set",
    "y_train",
    "y_val",
    "ytest_set",
]
for ddf in splits_df:
    globals()[ddf] = reduce_memory_usage(pd.DataFrame(globals()[ddf] ), verbose=True)

In [ ]:
# Factores que influyen en esta decisión


In [ ]:
cv_df=perform_cross_validation(modelos, X_train, y_train)

# Elegir hiperparámetros: Fine tuning



In [ ]:
# Según el volumen de datos y sus tipos


In [ ]:
results_df = hyperparameter_tuning(models_df, X_train, y_train)

In [ ]:
best_params=results_df.loc[results_df['Best Score']==max(results_df['Best Score']),'Best Parameters'].to_dict()[0]
best_model_name=results_df.loc[results_df['Best Score']==max(results_df['Best Score']),'Model'].to_dict()[0]
best_model=modelos.get(best_model_name)

In [ ]:
model_tuned=best_model.set_params(**best_params)
model_tuned.fit(X_train,y_train)
y_pred=model_tuned.predict(X_val)
recall_score(y_val,y_pred,average='macro')

# Validación


In [ ]:
# Dependerá de cada modelo. Ejecutamos


In [ ]:
plot_confusion_matrix(y_val, y_pred)

In [ ]:
cross_validation_with_confusion_matrix(model_tuned, X_val, y_val, nsplits=10)

# Prestaciones del modelo


In [ ]:
Classification_report = pd.DataFrame.from_dict(classification_report(y_val, y_pred, output_dict=True)).T
Classification_report

### ROC Curve y Area Under the Curve (AUC)

In [ ]:
# Llama a la función con tu modelo y datos de validación
plot_roc_curve(model_tuned, X_val, y_val)


In [ ]:
generate_roc_auc(model_tuned, X_train, y_train, X_val, y_val)

## Análisis de overfitting

In [ ]:

test_size_coef=0.2
cv=5
train_sizes=np.linspace(0.1, 1.0, 10)
scoring='recall_macro'

In [ ]:
t_size=0.15
perc_pca=0.95
thresh_norm=0.05
steps=5
score='recall_macro'

In [ ]:

train_sizes, train_scores, test_scores=learning_curve(model_tuned, X_train, y_train, cv=ShuffleSplit(n_splits=50, test_size=t_size, random_state=0), n_jobs=-1, train_sizes=np.linspace(0.1, 1.0,steps), scoring=score)
display = LearningCurveDisplay(train_sizes=train_sizes,
    train_scores=train_scores, test_scores=test_scores, score_name=score)
display.plot()
plt.show()

# 21. Bonus Track